# Домашнее задание к лекции "Основы веб-скрапинга"
## Обязательная часть
Вам необходимо написать функцию, которая будет основана на **поиске** по сайту [habr.com](https://habr.com/ru/search/).  
Функция в качестве параметра должна принимать **список** запросов для поиска (например, `['python', 'анализ данных']`) и на основе материалов, попавших в результаты поиска по **каждому** запросу, возвращать датафрейм вида:  

`<дата> - <заголовок> - <ссылка на материал>  `

В рамках задания предполагается работа только с одной (первой) страницей результатов поисковой выдачи для каждого запроса. Материалы в датафрейме не должны дублироваться, если они попадали в результаты поиска для нескольких запросов из списка.  

## Дополнительная часть (необязательная)
Функция из обязательной части задания должна быть расширена следующим образом:,  
*  кроме списка ключевых слов для поиска необходимо объявить параметр с количеством страниц поисковой выдачи. Т.е. при передаче в функцию аргумента `4` необходимо получить материалы с первых 4 страниц результатов;,
* в датафрейме должны быть столбцы с полным текстом найденных материалов и количеством лайков:  
`
<дата> - <заголовок> - <ссылка на материал> - <текст материала> - <количество лайков>
`
#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой [Google Colab](https://colab.research.google.com/).  
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.
   
Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.
    
##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам?
Прежде чем задать вопрос необходимо попробовать найти ответ самому в  интернете.  
Навык самостоятельного поиска информации – один из важнейших, и каждый практикующий специалист любого уровня это делает каждый день.  
Любой вопрос должен быть сформулирован по алгоритму:  
1. Что я делаю?  
1. Какого результата я ожидаю?  
1. Как фактический результат отличается от ожидаемого?  
1. Что я уже попробовал сделать, чтобы исправить проблему?  
   
По возможности, прикрепляйте к вопросу скриншоты, либо ссылки на код. Оставляйте только проблемный и воспроизводимый участок кода, все решение выкладывать не допускается.


In [1]:
import pandas as pd
import time
import requests 
import random 
import urllib
# !pip install lxml   
# import html5lib

try:
    from bs4 import BeautifulSoup
except ImportError:
    !pip install bs4
    from bs4 import BeautifulSoup

# try:
#     from selenium import webdriver
# except ImportError:
#     !pip install selenium
#     from selenium import webdriver


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ------------------------------------ 128.2/128.2 kB 501.8 kB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1264 sha256=44362621b38ef3490531c00f38d5a0915fd0df8b06260e3730c32ecc94fb669e
  Stored in directory: d:\users\max\appdata\local\pip\cache\wheels\e4\62\1d\d4d1bc4f33350ff84227f89b258edb552d604138e3739f5c83
Successfully built bs4


In [2]:
# response.encoding
# r.encoding = 'ISO-8859-1'

In [3]:
# soup = BeautifulSoup(response.text)#, 'html.parser')  # Batteries included, Decent speed, Lenient (As of Python 3.2)
# soup = BeautifulSoup(response.text, "html5lib") # Extremely lenient, Parses pages the same way a web browser does, Creates valid HTML5.  But! very slow
# soup = BeautifulSoup(response.text, "lxml") # Very fast, Lenient

In [4]:
# driver = webdriver.Chrome( executable_path= 'r\\che-ds\CS_IgSor\GIT_syno\chromedriver.exe')
# driver = webdriver.Chrome( executable_path= '/content/chromedriver.exe')
# driver.get('https://habr.com/ru/search/')
# content = diver.page_source
# driver.quit

In [5]:
# len(soup)

In [6]:
# params = {'q':'анализ данных', 'target_type': 'posts', 'order': 'relevance'} 
# print(urllib.parse.urlencode(params, quote_via=urllib.parse.quote) )

In [7]:
def habr_search(search_str, df, url):

# define an url parameters
    params = {'q':search_str, 'target_type': 'posts', 'order': 'relevance' }    
    # Convert a mapping object or a sequence of two-element tuples to a “percent-encoded” string
    # 'анализ данных' ->  'q=%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7%20%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85&target_type=posts&order=relevance'
    params = urllib.parse.urlencode(params, quote_via=urllib.parse.quote)    
    try:
# send a request
        req = requests.get(url, params= params)
    except: 
        print(f'Search: {search_str} failed')

# make pause
    time.sleep(random.random()*0.5)   
# feed the result into a soup
    soup = BeautifulSoup(req.text)

# find apropriate tag and class in HTML
    ntitle,ndate, nlink, nvotes, nlikes, ntext = ['']*6
    for element in soup.find_all('article', 'tm-articles-list__item'):
        if element.find('a', 'tm-article-snippet__title-link'):
            ntitle = element.find('a', 'tm-article-snippet__title-link').text
        if element.find('span','tm-article-snippet__datetime-published'):
            ndate = element.find('span','tm-article-snippet__datetime-published').text
        if element.find('a', class_='tm-article-snippet__title-link'):
            nlink = element.find('a', class_='tm-article-snippet__title-link').get('href')
        if element.find('span', class_='tm-votes-meter__value tm-votes-meter__value tm-votes-meter__value_positive tm-votes-meter__value_appearance-article tm-votes-meter__value_rating'):
            nvotes= element.find('span', class_='tm-votes-meter__value tm-votes-meter__value tm-votes-meter__value_positive tm-votes-meter__value_appearance-article tm-votes-meter__value_rating').text
        if element.find('span',class_='tm-icon-counter__value'):
            nlikes= element.find('span',class_='tm-icon-counter__value').text

# full article text search
        ntext = fulltext_search(link= SITE_URL+nlink)

        row = {'date': ndate, 
               'title': ntitle, 
               'link': SITE_URL + nlink, 
               'votes':nvotes,
               'likes': nlikes, 
               'full text': ntext }   
# append row to df 
        df = pd.concat(([df, pd.DataFrame([row])]) )

    return df

In [8]:
def fulltext_search(link, ):
    """ HABR article full text search """

    # follow the link
    resp = requests.get(link).text
    # A variant
    ntext = BeautifulSoup(resp).find('div',\
        "article-formatted-body article-formatted-body article-formatted-body_version-1")

    # has found
    if ntext:
        # A variant
        ntext= ntext.text
    else:
        # B variant
        ss = BeautifulSoup(resp).find('div',\
        "article-formatted-body article-formatted-body article-formatted-body_version-2")
        
        # if found
        if ss:
            str_=''
            # look for all entities in sub tree 
            for el in ss.contents:
                # try to extract text information from current <div>
                try: 
                    # if text attr exists
                    if el.text:
                        str_ += '\n' + el.text
                    ntext = str_
                except:
                    pass
    return ntext

In [9]:
tic = time.perf_counter()
SITE_URL = 'https://habr.com'

search_list = ['python', 'анализ данных']
# search_list = ['python']

habr_df = pd.DataFrame()
npages = 4
# iterate through search_list
for search_str in search_list:

    # depth of search
    for page in range(1,npages+1):
        if page == 1:
            habr_df = habr_search(search_str, habr_df, \
                                  'https://habr.com/ru/search/')
        else:
            habr_df = habr_search(search_str, habr_df, \
                                'https://habr.com/ru/search/page'+ str(page) )

# reset index to avoid zero indexes (and drop index column)    
habr_df.reset_index(drop=True, inplace= True)
print(f'Scraping completed  in {time.perf_counter() - tic:0.1f} s')
# habr_df.sort_values(by='votes', ascending= False)
habr_df

Scraping completed  in 137.0 s


,date,title,link,votes,likes,full text
0,20 января в 18:37,Курс «Python для инженеров». Старт 3 потока 31...,https://habr.com/ru/company/southbridge/news/t...,+10,7.1K,"\nКурс нацелен дать максимальную пользу, поэто..."
1,13 декабря 2021 в 09:00,Жаждущим автоматизации: открытый урок «ChatOps...,https://habr.com/ru/company/southbridge/news/t...,+9,1.8K,\n21 декабря Слёрм проведёт открытый урок «Cha...
2,21 апреля 2020 в 18:35,"Вышел Python 2.7.18, последний релиз ветки Pyt...",https://habr.com/ru/news/t/498364/,+19,8.5K,"\r\n20 апреля 2020 года, спустя почти десять л..."
3,6 июля 2021 в 13:29,Python Community Meetup 8/07: видео и материал...,https://habr.com/ru/company/raiffeisenbank/new...,+3,2.3K,"\nПервый открытый онлайн-митап сообщества, для..."
4,13 января в 18:35,Открытый урок «Пишем Custom Prometheus Exporte...,https://habr.com/ru/company/southbridge/news/t...,+10,2.4K,\n19 января Слёрм проведёт открытый урок «Пише...
...,...,...,...,...,...,...
155,24 июня 2016 в 23:12,"Достучаться до госорганов или что делать, если...",https://habr.com/ru/company/infoculture/blog/3...,+14,12K,\nисточник картинки: southriverrestoration.com...
156,24 апреля 2017 в 13:46,Приглашаем на IV конференцию по практическому ...,https://habr.com/ru/company/flyelephant/blog/3...,+22,2.3K,\n\r\nКоманда FlyElephant приглашает всех 13 м...
157,10 июля 2017 в 16:14,Лекция о двух библиотеках Яндекса для работы с...,https://habr.com/ru/company/yandex/blog/332688/,+33,12K,Пару недель назад в Яндексе прошла встреча PyD...
158,22 октября 2018 в 13:29,Бизнес на персональных данных: как добиться ус...,https://habr.com/ru/company/digitalrightscente...,+7,7.1K,\n\r\n«Данные – нефть цифровой экономики» — вы...
